In [7]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions

# from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import mysql.connector as cnt
import random
import setuptools.dist
import undetected_chromedriver as uc

In [15]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(1)
            db.commit()
            time.sleep(2)
    c.close()
    db.close()
    return True


def createMainTable(dbname:str) -> bool:
    sql = '''
    drop table if exists TeleVarle
    '''
    db = sqlite3.connect(dbname)
    c = db.cursor()
    c.execute(sql)
    sql = '''create table if not exists TeleVarle
    (
        nuoroda text,
        firstInfo text
    )
    '''
    c.execute(sql)
    c.close()
    db.close()
    return True
    pass

def writeMany(lst, table, dbname) -> bool:
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''insert into {table} values ({n})'''
    q = ['?' for i in range(0,len(lst))]
    q_ = ', '.join(q)
    r_ = sql.format(table=table, n=q_)
    # print('::PROVIDED::', lst, len(lst), '::SQL COMMAND::', r_, len(q), sep='\n')
    c.execute(r_, lst)
    db.commit()
    c.close()
    db.close()
    return True


In [18]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
url = 'https://www.varle.lt/televizoriai/?p=1'
driver.get(url)
time.sleep(5)
source = driver.page_source


driver.close()
print('Baigta!!!')

Baigta!!!


In [19]:
bs = BeautifulSoup(source, 'html.parser')
p_nuorodos = []

linkai = bs.find_all('div', {'class':'GRID_ITEM'})
for linkas in linkai:
    # linkas = bs.find('div', {'class':'product-title'}).find('div', {'class':'product-title'}).find('a')
    link = linkas.find('div', {'class':'product-title'}).find('a')
    # print(link['href'])
    add_link = 'https://www.varle.lt'+ link['href']
    # print(add_link)
    p_nuorodos.append(add_link)
print(len(p_nuorodos))
print(p_nuorodos)

pages = []
pages_numbers = bs.find('div', {'class':'PAGINATION'}).find_all('li', {'class':'number'})
for page in pages_numbers:
    # print(page.text.strip())
    pages.append(int(page.text.strip()))
pages_max_number = max(pages)
print(pages_max_number)

38
['https://www.varle.lt/televizoriai/televizorius-lg-oled55b42la--35671867.html', 'https://www.varle.lt/televizoriai/televizorius-lg-oled55c41la--35999335.html', 'https://www.varle.lt/televizoriai/televizorius-samsung-qe50q60dauxxh--33548199.html', 'https://www.varle.lt/televizoriai/televizorius-samsung-qe55q60dauxxh--33548188.html', 'https://www.varle.lt/televizoriai/televizorius-lg-oled65c41la-65-139-cm-oled-evo--35999814.html', 'https://www.varle.lt/televizoriai/televizorius-samsung-qe43q60dauxxh--33548195.html', 'https://www.varle.lt/televizoriai/televizorius-samsung-tv-uhd-43-ue43cu8072u--23626657.html', 'https://www.varle.lt/televizoriai/televizorius-xiaomi-tv-a-2025-43-smart-tv--36249643.html', 'https://www.varle.lt/televizoriai/televizoriussamsung434ksmart3840x2160belaidis--33548172.html', 'https://www.varle.lt/televizoriai/telefunken-42fg6010-fhd-televizorius--21189629.html', 'https://www.varle.lt/televizoriai/televizorius-toshiba-32wl1c63dg--16606805.html', 'https://www.var

In [20]:
for nuoroda in p_nuorodos[:1]:
    print(nuoroda)
    opcijos = Options()
    opcijos.add_argument('--incognito')
    opcijos.add_argument('--ignore-certificate-errors')
    opcijos.add_argument('--ignore-ssl-errors')
    driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
    # url = nuoroda
    url = 'https://www.varle.lt/televizoriai/televizorius-xiaomi-tv-a-2025-32-smart-tv--36963281.html'
    driver.get(url)
    time.sleep(5)
    source = driver.page_source


    driver.quit()

https://www.varle.lt/televizoriai/televizorius-lg-oled55b42la--35671867.html


In [21]:
bs = BeautifulSoup(source, 'html.parser')
pavadinimas = bs.find('div', {'class':'main'}).find('h1', {'class':'title'}).text.strip()
print(pavadinimas)

# varle_kodas = bs.find('div', {'class':'main'}).find('span', {'itemprop':'sku'}).text
# print(varle_kodas)

kaina = bs.find('span', {'class':'price-value'}).text.strip()
print(kaina)

buvusi_kaina = bs.find('span', {'class':'previous-price'})
if buvusi_kaina != None:
    buvusi_kaina = buvusi_kaina.text.strip()
else:
    buvusi_kaina = None
print(buvusi_kaina)

specifiacija_keys = []
specifiacija_values = []
try:
    specifiacija = bs.find('div', {'class':'PRODUCT_SPEC'}).find_all('li', {'class':'spec-line'})
    for line in specifiacija:
        try:
            spec_key = line.find('span', {'class':'left-column'}).text
            # print(spec_key)
            specifiacija_keys.append(spec_key)
            spec_value = line.find('span', {'class':'right-column'}).text
            # print(spec_value)
            specifiacija_values.append(spec_value)
        except:
            pass
        try:
            spec_key = line.find('div', {'class':'left-column'}).text
            # print(spec_key)
            specifiacija_keys.append(spec_key)
            spec_value = line.find('div', {'class':'right-column'}).text
            # print(spec_value)
            specifiacija_values.append(spec_value)
        except:
            pass
except:
    pass

try:
    specifiacija_k = bs.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'key'})
    for key in specifiacija_k:
        # print(key.text.strip())
        specifiacija_keys.append(key.text.strip().lower())
    specifiacija_v = bs.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'value'})
    for val in specifiacija_v:
        # print(val.text.strip())
        specifiacija_values.append(val.text.strip())       
except:
    pass

print(len(specifiacija_keys), specifiacija_keys)
print(len(specifiacija_values), specifiacija_values)

spec_filtrai_keys = []
spec_filtrai_values = []
try:
    spec_filtai = bs.find('div', {'class':'PRODUCT_SPEC_FILTERS'}).find_all('li', {'class':'spec-line'})
    for line in spec_filtai:
        spec_key = line.find('span', {'class':'left-column'}).text.strip().lower()
        # print(spec_key)
        spec_filtrai_keys.append(spec_key)
        spec_value = line.find('span', {'class':'right-column'}).text.strip()
        # print(spec_value)
        spec_filtrai_values.append(spec_value)
except:
    pass
    
print(len(spec_filtrai_keys), spec_filtrai_keys)
print(len(spec_filtrai_values), spec_filtrai_values)

# for k,v in zip(specifiacija_keys, specifiacija_values):
#     print(k, v)

Televizorius Xiaomi TV | A 2025 | 32 | Smart TV | Google TV | HD | Juodas
147.99
169 €
44 ['Gamintojas', 'Modelis', 'Valdymas balsu', 'Tvx08 procesorius', 'Ekrano tipas', 'Garso sistema', 'Svoris su stovu', 'Tvx09 wi-fi', 'Išmanusis nuotolinio valdymo pultas', 'Tvx03 atnaujinimo dažnis', '"ambilight"', 'Smart TV technologija', 'Tvx07 operacinė sistema', 'Matmenys su stovu', 'Didelis dinaminis diapazonas (hdr)', 'Matmenys be stovo (pxaxd)', 'Tvx12 usb', 'Tvx10 bluetooth', 'Miracast technologija', 'Energijos suvartojimas aktyviu režimu', 'Išmaniosios televizijos platforma', 'Metinis energijos suvartojimas', 'Televizijos patarimai', 'Svoris be stovo', 'Energijos suvartojimo klasė', 'Tvx02 ekrano dydis (cm)', 'Micro dimming technologija', 'Tvx11 nuotolinio valdymo pultas', 'Garso galia', 'Tvx14 pilno dydžio', 'Energijos sąnaudos (1000 valandų)', 'Valdymas išmaniųjų įrenginių pagalba', 'Wi-fi palaikymas', 'Tvx04 ekrano skiriamoji geba', 'Tvx13 hdmi', 'Derintuvas', 'Energijos etiketės tipas'

In [ ]:
# createMainTable('VarlePigu.db')


True

In [ ]:
# Televiroziai Varle scraper

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
url = 'https://www.varle.lt/televizoriai/?p=1'
driver.get(url)
time.sleep(5)
source = driver.page_source

bs = BeautifulSoup(source, 'html.parser')
pages = []
pages_numbers = bs.find('div', {'class':'PAGINATION'}).find_all('li', {'class':'number'})
for page in pages_numbers:
    # print(page.text.strip())
    pages.append(int(page.text.strip()))
pages_max_number = max(pages)

counts = 1
psl = 1
# iki_kokio_psl = 3
# while psl < iki_kokio_psl:
while psl < pages_max_number + 1:
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    p_nuorodos = []

    linkai = bs.find_all('div', {'class':'GRID_ITEM'})
    for linkas in linkai:
        # linkas = bs.find('div', {'class':'product-title'}).find('div', {'class':'product-title'}).find('a')
        link = linkas.find('div', {'class':'product-title'}).find('a')
        # print(link['href'])
        add_link = 'https://www.varle.lt'+ link['href']
        # print(add_link)
        p_nuorodos.append(add_link)
        
    for nuoroda in p_nuorodos:
        print(psl, nuoroda)
        url = nuoroda
        driver.get(url)
        # time.sleep(random.randint(5,10))
        time.sleep(4)
        source = driver.page_source
        
        cols_i_have = getColNamesList(table='TeleVarle', dbname='VarlePigu.db')
        
        duomenys = {}
        for col in cols_i_have:
            duomenys[col] = None
            
        duomenys['nuoroda'] = nuoroda
        
        bs_preke = BeautifulSoup(source, 'html.parser')
        
        firstInfo = bs_preke.find('div', {'class':'main'}).find('h1', {'class':'title'}).text.strip()
        duomenys['firstInfo'] = firstInfo
        # print(pavadinimas)


        kaina = bs_preke.find('span', {'class':'price-value'}).text.strip()
        duomenys['kaina'] = kaina
        # print(kaina)

        buvusi_kaina = bs_preke.find('span', {'class':'previous-price'})
        if buvusi_kaina != None:
            buvusi_kaina = buvusi_kaina.text.strip()
        else:
            buvusi_kaina = None
        duomenys['buvusiKaina'] = buvusi_kaina
        # print(buvusi_kaina)

        specifiacija_keys = []
        specifiacija_values = []
        try:
            specifiacija = bs_preke.find('div', {'class':'PRODUCT_SPEC'}).find_all('li', {'class':'spec-line'})
            for line in specifiacija:
                try:
                    spec_key = line.find('span', {'class':'left-column'}).text.lower()
                    # print(spec_key)
                    specifiacija_keys.append(spec_key)
                    spec_value = line.find('span', {'class':'right-column'}).text
                    # print(spec_value)
                    specifiacija_values.append(spec_value)
                except:
                    pass
                try:
                    spec_key = line.find('div', {'class':'left-column'}).text.lower()
                    # print(spec_key)
                    specifiacija_keys.append(spec_key)
                    spec_value = line.find('div', {'class':'right-column'}).text
                    # print(spec_value)
                    specifiacija_values.append(spec_value)
                except:
                    pass
        except:
            pass

        try:
            specifiacija_k = bs_preke.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'key'})
            for key in specifiacija_k:
                # print(key.text.strip())
                specifiacija_keys.append(key.text.strip().lower())
            specifiacija_v = bs_preke.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'value'})
            for val in specifiacija_v:
                # print(val.text.strip())
                specifiacija_values.append(val.text.strip())       
        except:
            pass
        
        
        # print(len(specifiacija_keys), specifiacija_keys)
        # print(len(specifiacija_values), specifiacija_values)

        spec_filtrai_keys = []
        spec_filtrai_values = []
        try:
            spec_filtai = bs_preke.find('div', {'class':'PRODUCT_SPEC_FILTERS'}).find_all('li', {'class':'spec-line'})
            for line in spec_filtai:
                spec_key = line.find('span', {'class':'left-column'}).text.strip().lower()
                # print(spec_key)
                spec_filtrai_keys.append(spec_key)
                spec_value = line.find('span', {'class':'right-column'}).text.strip()
                # print(spec_value)
                spec_filtrai_values.append(spec_value)
                
            # print(len(spec_filtrai_keys), spec_filtrai_keys)
            # print(len(spec_filtrai_values), spec_filtrai_values)
        except:
            pass
        
        for k, v in zip(spec_filtrai_keys, spec_filtrai_values):
            duomenys[k] = v
            
        for k, v in zip(specifiacija_keys, specifiacija_values):
            duomenys[k] = v
            
        cols_i_need = list(duomenys.keys())
        
        data = tuple(duomenys.values())
        
        cols_i_have = getColNamesList(table='TeleVarle', dbname='VarlePigu.db')
        
        alterTableAddColumn(cols_i_need, cols_i_have, 'TeleVarle', 'VarlePigu.db')
        
        writeMany(data, 'TeleVarle', 'VarlePigu.db')
        print(f'Įrašytas įrašas Nr {counts}, puslapis {psl}')
        counts = counts +1
    
    print(f'Įrašytas puslapis {psl}')
    psl = psl +1
    # if psl < iki_kokio_psl:
    if psl <= pages_max_number:
        next_nuoroda = f'https://www.varle.lt/televizoriai/?p={psl}'
        driver.get(next_nuoroda)
        time.sleep(5)
        print(f'kitas puslapis: {psl}')
    
driver.quit()
print('Baigta!!!')


1 https://www.varle.lt/televizoriai/televizorius-lg-oled55b42la--35671867.html
Įrašytas įrašas Nr 1, puslapis 1
1 https://www.varle.lt/televizoriai/televizorius-lg-oled55c41la--35999335.html
Įrašytas įrašas Nr 2, puslapis 1
1 https://www.varle.lt/televizoriai/televizorius-samsung-qe50q60dauxxh--33548199.html
Įrašytas įrašas Nr 3, puslapis 1
1 https://www.varle.lt/televizoriai/televizorius-samsung-qe55q60dauxxh--33548188.html
Įrašytas įrašas Nr 4, puslapis 1
1 https://www.varle.lt/televizoriai/televizorius-lg-oled65c41la-65-139-cm-oled-evo--35999814.html
Įrašytas įrašas Nr 5, puslapis 1
1 https://www.varle.lt/televizoriai/televizorius-samsung-qe43q60dauxxh--33548195.html
Įrašytas įrašas Nr 6, puslapis 1
1 https://www.varle.lt/televizoriai/televizorius-samsung-tv-uhd-43-ue43cu8072u--23626657.html
Įrašytas įrašas Nr 7, puslapis 1
1 https://www.varle.lt/televizoriai/televizorius-xiaomi-tv-a-2025-43-smart-tv--36249643.html
Įrašytas įrašas Nr 8, puslapis 1
1 https://www.varle.lt/televizoriai

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=52613): Read timed out. (read timeout=120)